## code 1

In [1]:
import time
import os
import sys

import pybullet as p
#from manipulatorsMOD import SawyerMOD  

from cairo_simulator.core.utils import ASSETS_PATH
from cairo_simulator.core.log import Logger
from cairo_simulator.core.link import *
from cairo_simulator.core.simulator import Simulator, SimObject
from cairo_simulator.devices.manipulators import Sawyer
#from cairo_simulator.devices.manipulatorsMOD import SawyerMOD #New python file that allows a modified sawyer robot to be used (ie, a sawyer holding a block)
from manipulatorsMOD import SawyerMOD
from utils import load_configuration, save_config_to_configuration_file, manual_control, create_cuboid_obstacle

from pybullet_utils import bullet_client as bc
from pybullet_utils import urdfEditor as ed
import pybullet_data
import time


def main():

    use_ros = False
    use_real_time = True
    logger = Logger()
    sim = Simulator(logger=logger, use_ros=use_ros, use_real_time=use_real_time) # Initialize the Simulator
    p.setGravity(0,0,-9.81)
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,0)
    p.setPhysicsEngineParameter(enableFileCaching=0)
    ground_plane = SimObject("Ground", "plane.urdf", [0,0,0])

    table = SimObject('table', 'NEWtable.urdf',  (0.9, 0.1, .47),(1.5708*2,0,0),fixed_base=1)
    #print(p.getNumJoints(table.get_simulator_id()))
    
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
    
    block = SimObject('block1', '1block.urdf', (0.69, -0.2, .55))
    sim_obj1 = SimObject('hole1', '1.5hole.urdf',  (0.69, 0.1, .530),(0,0,0),fixed_base=0)  #1.5708 for 90 deg rotation
    sim_obj2 = SimObject('hole2', '1.25hole.urdf', (0.69, 0.3, .530),(0,0,0),fixed_base=0)
    sim_obj3 = SimObject('hole3', '1.15hole.urdf', (0.69, 0.5, .530),(0,0,0),fixed_base=0)    
    
    #___________________#Test Combining URDF's
    #https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/combineUrdf.py
    p0 = bc.BulletClient(connection_mode=p.DIRECT)
    p0.setAdditionalSearchPath(pybullet_data.getDataPath())
    p1 = bc.BulletClient(connection_mode=p.DIRECT)
    p1.setAdditionalSearchPath(pybullet_data.getDataPath())

    kuka = p0.loadURDF('1.5hole.urdf')
    #husky = p1.loadURDF('1block.urdf', flags=p0.URDF_USE_IMPLICIT_CYLINDER)  #this flag appears to have no impact
    husky = p1.loadURDF('1block.urdf')
    
    ed0 = ed.UrdfEditor()
    ed0.initializeFromBulletBody(husky, p1._client)
    ed1 = ed.UrdfEditor()
    ed1.initializeFromBulletBody(kuka, p0._client)

    parentLinkIndex = 0

    jointPivotXYZInParent = [0, 0, 0]
    jointPivotRPYInParent = [0, 0, 0]

    jointPivotXYZInChild = [0, 0, 0]
    jointPivotRPYInChild = [0, 0, 0]

    #link index = joint index
    newjoint = ed0.joinUrdf(ed1, parentLinkIndex, jointPivotXYZInParent, jointPivotRPYInParent,
                            jointPivotXYZInChild, jointPivotRPYInChild, p0._client, p1._client)
    newjoint.joint_type = p0.JOINT_FIXED

    ed0.saveUrdf("combined.urdf")
    sim_obj3 = SimObject('hole3',"combined.urdf", (0.69, -0.4, .530),(0,0,0),fixed_base=0)  
    
    #_______________________ end URDF combining test
    
    
    #print(ASSETS_PATH)
    robotfilelocation=ASSETS_PATH + 'sawyer_description/urdf/sawyer_static.urdf'
    sawyer_robot = Sawyer("sawyer0", [0, 0, 0.8], fixed_base=1)
    #sawyer_robot = SawyerMOD("sawyer0",robotfilelocation, [0, 0, 0.8], fixed_base=1)
    
    
    #___________________#Modifying Sawyer URDF's
    #edited from https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/combineUrdf.py
    p3 = bc.BulletClient(connection_mode=p.DIRECT)
    p3.setAdditionalSearchPath(pybullet_data.getDataPath())
    p4 = bc.BulletClient(connection_mode=p.DIRECT)
    p4.setAdditionalSearchPath(pybullet_data.getDataPath())

    child = p3.loadURDF('1block.urdf')
    parent = p4.loadURDF(robotfilelocation)
    
    ed_child = ed.UrdfEditor()
    ed_child.initializeFromBulletBody(child, p3._client)
    ed_parent = ed.UrdfEditor()
    ed_parent.initializeFromBulletBody(parent, p4._client)
    
    #link index = joint index
   
    parentLinkIndex2 =26 #'right_l6'
    #get links  get the one for fingertip

    jointPivotXYZInParent = [0, 0, 0]
    jointPivotRPYInParent = [0, 0, 0]
    jointPivotXYZInChild = [0, 0, 0]
    jointPivotRPYInChild = [0, 0, 0]

    newjoint2 = ed_parent.joinUrdf(ed_child, parentLinkIndex2, jointPivotXYZInParent, jointPivotRPYInParent,
                            jointPivotXYZInChild, jointPivotRPYInChild, p3._client, p4._client)
    newjoint2.joint_type = p3.JOINT_FIXED
    ed_parent.saveUrdf("combinedRobot2.urdf")

    robotfilelocation2='/home/scott/ScheragaThesis/combinedRobot2.urdf'
    #robotfilelocation2='combinedRobot2.urdf'
    print("files:")
    print(robotfilelocation)
    print(robotfilelocation2)
    
    #p.getJointInfo(self._simulator_id, i)  #to get joint
    
    #sawyer_robot = SawyerMOD("sawyer0",robotfilelocation, [0, 0, 0.8], fixed_base=1)
    
    #_______________________
    
    #robotID=sawyer_robot.get_simulator_id() #numeric code for robot
    #print(robotID)  #robot id is 7
    #print(sawyer_robot._arm_dof_indices) #[3, 8, 9, 10, 11, 13, 16]  #these correspond to the links
    #print(sawyer_robot._arm_dof_names) #['right_j0', 'right_j1', 'right_j2', 'right_j3', 'right_j4', 'right_j5', 'right_j6']
    #print(get_joint_info_by_name(7,"right_j6"))
    
    #to display all joint information
    """
    dofindex=[3, 8, 9, 10, 11, 13, 16]
    for x in dofindex:
        print(get_joint_info(7,x))
        print("  ")
    """
    
#     for j in range(p.getNumJoints(sawyer_robot, physicsClientId=self.id)):
#             joint_info = p.getJointInfo(sawyer_robot, j, physicsClientId=self.id)
#             print(joint_info)
            
            #joint_name = joint_info[1]
            #joint_pos = joint_positions[j]
            #lower_limit = joint_info[8]
            #upper_limit = joint_info[9]
   

    #enable force/torque sensors at all joints
    """
    dofindex=[3, 8, 9, 10, 11, 13, 16]
    for x in dofindex:
        p.enableJointForceTorqueSensor(7,x,enableSensor=1)  #args: bodyID#, jointIndex,enablesensor
    """
    
    orientation= [0,1,0,0] #quaternion hand pointing down..ish
    xoffset=0
    yoffset=0
    
    #initial positioning of arm
    joint_config2 = [0.69+xoffset, 0.1+yoffset, 1]
    joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
    sawyer_robot.move_to_joint_pos(joint_config)
    time.sleep(1.)
    joint_config2 = [0.69+xoffset, 0.1+yoffset, 0.8]
    joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
    sawyer_robot.move_to_joint_pos(joint_config)
    
    key_w = ord('u') #y up
    key_a = ord('h') #x down
    key_s = ord('j') #y down
    key_d = ord('k') #x up
    key_o = ord('o') #z up
    key_l = ord('l') #z down
    key_8 = ord('8') #print all 6dof force/torque info
    key_9 = ord('9') #print info
    
    loop_count = 0 # Loop until someone shuts us down
    xcurrent=0.69
    ycurrent=0.1
    zcurrent=0.8
    # the center of 1.5 hole is roughly 0.71  0.08 OR   ~0.78 in x and y
    #print("jointPosition,jointVelocity,(6dof)jointReactionForces, appliedJointMotorTorque")
    for i in range(p.getNumJoints(7)):
        j_info = p.getJointInfo(7, i)
        print(j_info)
        
    print("HK to jog_X         UJ to jog_Y           OL to jog_Z")
    print("8 for all joint force/torque, 9 for J16 and XYZ location")
    while True:
            xoffset=0
            yoffset=0
            
            keys = p.getKeyboardEvents()
            if key_w in keys and keys[key_w] & p.KEY_WAS_TRIGGERED:
                ycurrent+=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_a in keys and keys[key_a] & p.KEY_WAS_TRIGGERED:
                xcurrent-=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_s in keys and keys[key_s] & p.KEY_WAS_TRIGGERED:
                ycurrent-=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_d in keys and keys[key_d] & p.KEY_WAS_TRIGGERED:
                xcurrent+=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_o in keys and keys[key_o] & p.KEY_WAS_TRIGGERED:
                zcurrent+=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_l in keys and keys[key_l] & p.KEY_WAS_TRIGGERED:
                zcurrent-=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)    
                
            if key_8 in keys and keys[key_8] & p.KEY_WAS_TRIGGERED:
                print(" ")
                dofindex=[3, 8, 9, 10, 11, 13, 16]
                print("All JointReactionForces")
                for x in dofindex:
                    jointinfo=p.getJointState(7,int(x)) 
                    rounded6dof = [round(num, 3) for num in jointinfo[2]]
                    print("J", x , " ", rounded6dof) 
                
            if key_9 in keys and keys[key_9] & p.KEY_WAS_TRIGGERED:
                print(" ")
                print("location", xcurrent,"  ",ycurrent,"  ",zcurrent)
                #print("enter joint number [3, 8, 9, 10, 11, 13, 16]")
                #a = input()
                #jointinfo=p.getJointState(7,16) 
                #jointinfo=p.getJointState(7,int(a))
                jointinfo=p.getJointState(7,16) 
                rounded6dof = [round(num, 3) for num in jointinfo[2]]
                print("J 16",rounded6dof)
                #print(jointinfo)
                
            time.sleep(0.2)
            
    #manual_control(sawyer_robot)    
    try:
        while True:
            sim.step()
    except KeyboardInterrupt:
        p.disconnect()
        sys.exit(0)

In [ ]:
main()


files:
/home/scott/ScheragaThesis/cairo_simulator/src/cairo_simulator/core/../../../assets/sawyer_description/urdf/sawyer_static.urdf
/home/scott/ScheragaThesis/combinedRobot2.urdf
(0, b'torso_t0', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'torso', (0.0, 0.0, 0.0), (0.1, 0.0, -0.07), (0.0, 0.0, 0.0, 1.0), -1)
(1, b'pedestal_fixed', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'pedestal', (0.0, 0.0, 0.0), (0.1, 0.0, -0.07), (0.0, 0.0, 0.0, 1.0), -1)
(2, b'right_arm_mount', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'right_arm_base_link', (0.0, 0.0, 0.0), (0.1, 0.0, -0.07), (0.0, 0.0, 0.0, 1.0), -1)
(3, b'right_j0', 0, 7, 6, 1, 0.0, 0.0, -3.0503, 3.0503, 80.0, 1.74, b'right_l0', (0.0, 0.0, 1.0), (0.0006241, 2.8025e-05, 0.014595999999999998), (0.0, 0.0, 0.0, 1.0), 2)
(4, b'head_pan', 0, 8, 7, 1, 0.0, 0.0, -5.1477, 0.9559, 8.0, 1.8, b'head', (0.0, 0.0, 1.0), (-0.024366, -0.010969, 0.15286999999999998), (0.0, 0.0, 0.0, 1.0), 3)
(5, b'display_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0

In [ ]:
for i in range(p.getNumJoints(7)):
        j_info = p.getJointInfo(7, i)
        print(j_info)
        
        

## code 2


In [ ]:
    #___________________#Combining URDF's
    #https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/combineUrdf.py
    p0 = bc.BulletClient(connection_mode=p.DIRECT)
    p0.setAdditionalSearchPath(pybullet_data.getDataPath())
    p1 = bc.BulletClient(connection_mode=p.DIRECT)
    p1.setAdditionalSearchPath(pybullet_data.getDataPath())

    #husky = p1.loadURDF('1block.urdf', flags=p0.URDF_USE_IMPLICIT_CYLINDER)  #this flag appears to have no impact
    husky = p1.loadURDF('1block.urdf')
    kuka = p0.loadURDF('1.5hole.urdf')

    ed0 = ed.UrdfEditor()
    ed0.initializeFromBulletBody(husky, p1._client)
    ed1 = ed.UrdfEditor()
    ed1.initializeFromBulletBody(kuka, p0._client)

    parentLinkIndex = 0

    jointPivotXYZInParent = [0, 0, 0]
    jointPivotRPYInParent = [0, 0, 0]

    jointPivotXYZInChild = [0, 0, 0]
    jointPivotRPYInChild = [0, 0, 0]

    newjoint = ed0.joinUrdf(ed1, parentLinkIndex, jointPivotXYZInParent, jointPivotRPYInParent,
                            jointPivotXYZInChild, jointPivotRPYInChild, p0._client, p1._client)
    newjoint.joint_type = p0.JOINT_FIXED

    ed0.saveUrdf("combined.urdf")
    #pgui = bc.BulletClient(connection_mode=p.GUI)
    #pgui.configureDebugVisualizer(pgui.COV_ENABLE_RENDERING, 0)
    
    orn = [0, 0, 0, 1]
    #ed0.createMultiBody([0, 0, 0], orn, pgui._client)  #doesn't work
    
    
    #ed0.createMultiBody([(0.69, -0.1, .530)], orn)  #works
    sim_obj3 = SimObject('hole3',"combined.urdf", (0.69, -0.4, .530),(0,0,0),fixed_base=0) 


0
1
p0.getNumBodies()= 1
p1.getNumBodies()= 1
fileNames= ['./1.5in squarehole_C.SLDPRT/meshes/1.5in squarehole_C.SLDPRT.STL']


 aaa


ehllo aaa
